# Distributed HPO with Ray

##### Training a simple XGBoost classifier without Ray

In [1]:
import sklearn.datasets
import sklearn.metrics
from sklearn.model_selection import train_test_split
import xgboost as xgb


def train_breast_cancer(config):
    # Load dataset
    data, labels = sklearn.datasets.load_breast_cancer(return_X_y=True)
    # Split into train and test set
    train_x, test_x, train_y, test_y = train_test_split(data, labels, test_size=0.25)
    # Build input matrices for XGBoost
    train_set = xgb.DMatrix(train_x, label=train_y)
    test_set = xgb.DMatrix(test_x, label=test_y)
    # Train the classifier
    results = {}
    bst = xgb.train(
        config,
        train_set,
        evals=[(test_set, "eval")],
        evals_result=results,
        verbose_eval=False,
    )
    return results


results = train_breast_cancer(
    {"objective": "binary:logistic", "eval_metric": ["logloss", "error"]}
)
accuracy = 1.0 - results["eval"]["error"][-1]
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.9301


##### Still Training with some parameters but without Ray

In [2]:
config = {
    "objective": "binary:logistic",
    "eval_metric": ["logloss", "error"],
    "max_depth": 2,
    "min_child_weight": 0,
    "subsample": 0.8,
    "eta": 0.2,
}
results = train_breast_cancer(config)
accuracy = 1.0 - results["eval"]["error"][-1]
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.9301


### What distributed HPO looks like with Ray

In [3]:
import sklearn.datasets
import sklearn.metrics

from ray import train, tune

In [4]:
def train_breast_cancer(config):
    # Load dataset
    data, labels = sklearn.datasets.load_breast_cancer(return_X_y=True)
    # Split into train and test set
    train_x, test_x, train_y, test_y = train_test_split(data, labels, test_size=0.25)
    # Build input matrices for XGBoost
    train_set = xgb.DMatrix(train_x, label=train_y)
    test_set = xgb.DMatrix(test_x, label=test_y)
    # Train the classifier
    results = {}
    xgb.train(
        config,
        train_set,
        evals=[(test_set, "eval")],
        evals_result=results,
        verbose_eval=False,
    )
    # Return prediction accuracy
    accuracy = 1.0 - results["eval"]["error"][-1]
    train.report({"mean_accuracy": accuracy, "done": True})

In [5]:
config = {
    "objective": "binary:logistic",
    "eval_metric": ["logloss", "error"],
    "max_depth": tune.randint(1, 9),
    "min_child_weight": tune.choice([1, 2, 3]),
    "subsample": tune.uniform(0.5, 1.0),
    "eta": tune.loguniform(1e-4, 1e-1),
}

In [6]:
tuner = tune.Tuner(
    train_breast_cancer,
    tune_config=tune.TuneConfig(num_samples=10),
    param_space=config,
)

In [7]:
import ray
try:
    ray.shutdown()
except:
    pass
ray.init(address="auto")

2024-04-24 03:15:01,053	INFO worker.py:1540 -- Connecting to existing Ray cluster at address: 10.244.21.11:6379...
2024-04-24 03:15:03,161	INFO worker.py:1715 -- Connected to Ray cluster. View the dashboard at http://10.244.21.11:8265 
[2024-04-24 03:15:03,164 I 8619 8619] logging.cc:230: Set ray log level from environment variable RAY_BACKEND_LOG_LEVEL to -1


Python version:,3.8.10
Ray version:,2.9.3
Dashboard:,http://10.244.21.11:8265


(raylet, ip=10.244.22.11) [2024-04-24 03:15:16,685 I 2942 2942] logging.cc:230: Set ray log level from environment variable RAY_BACKEND_LOG_LEVEL to -1
(raylet, ip=10.244.24.11) [2024-04-24 03:15:46,010 I 2996 2996] logging.cc:230: Set ray log level from environment variable RAY_BACKEND_LOG_LEVEL to -1
(train_breast_cancer pid=3058, ip=10.244.24.11) [03:15:48] WARNING: /workspace/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
(train_breast_cancer pid=3029, ip=10.244.23.11) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/home/artifacts/earlystopping/train_breast_cancer_2024-04-24_03-15-45/train_breast_cancer_f0d16_00003_3_eta=0.0930,max_depth=6,min_child_weight=3,subsample=0.7415_2024-04-24_03-15-45/checkpoint_000000)
(raylet) [2024-04-24 03:15:50,802 I 8919 8919] logging.cc:230: Set ray log level from environment variable RAY_BACKEND_LOG

In [8]:
results = tuner.fit()

2024-04-24 03:15:20,381	ERROR worker.py:405 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): The worker died unexpectedly while executing this task. Check python-core-worker-*.log files for more information.
2024-04-24 03:15:20,407	INFO tune.py:1042 -- Total run time: 6.15 seconds (4.69 seconds for the tuning loop).


### Early stopping in HPO

In [9]:
import sklearn.datasets
import sklearn.metrics
from ray.tune.schedulers import ASHAScheduler
from sklearn.model_selection import train_test_split
import xgboost as xgb

from ray import tune
from ray.tune.integration.xgboost import TuneReportCheckpointCallback

In [10]:
def train_breast_cancer(config: dict):
    # This is a simple training function to be passed into Tune
    # Load dataset
    data, labels = sklearn.datasets.load_breast_cancer(return_X_y=True)
    # Split into train and test set
    train_x, test_x, train_y, test_y = train_test_split(data, labels, test_size=0.25)
    # Build input matrices for XGBoost
    train_set = xgb.DMatrix(train_x, label=train_y)
    test_set = xgb.DMatrix(test_x, label=test_y)
    # Train the classifier, using the Tune callback
    xgb.train(
        config,
        train_set,
        evals=[(test_set, "eval")],
        verbose_eval=False,
        # `TuneReportCheckpointCallback` defines the checkpointing frequency and format.
        callbacks=[TuneReportCheckpointCallback(frequency=1)],
    )

In [11]:
def get_best_model_checkpoint(results):
    best_result = results.get_best_result()

    # `TuneReportCheckpointCallback` provides a helper method to retrieve the
    # model from a checkpoint.
    best_bst = TuneReportCheckpointCallback.get_model(best_result.checkpoint)

    accuracy = 1.0 - best_result.metrics["eval-error"]
    print(f"Best model parameters: {best_result.config}")
    print(f"Best model total accuracy: {accuracy:.4f}")
    return best_bst


In [12]:
def tune_xgboost(smoke_test=False):
    search_space = {
        # You can mix constants with search space objects.
        "objective": "binary:logistic",
        "eval_metric": ["logloss", "error"],
        "max_depth": tune.randint(1, 9),
        "min_child_weight": tune.choice([1, 2, 3]),
        "subsample": tune.uniform(0.5, 1.0),
        "eta": tune.loguniform(1e-4, 1e-1),
    }
    # This will enable aggressive early stopping of bad trials.
    scheduler = ASHAScheduler(
        max_t=10, grace_period=1, reduction_factor=2  # 10 training iterations
    )

    tuner = tune.Tuner(
        train_breast_cancer,
        tune_config=tune.TuneConfig(
            metric="eval-logloss",
            mode="min",
            scheduler=scheduler,
            num_samples=1 if smoke_test else 10,
        ),
        run_config=ray.train.RunConfig(
            storage_path="/home/artifacts/earlystopping/"
        ),
        param_space=search_space,
    )
    results = tuner.fit()
    return results

In [13]:
results = tune_xgboost(smoke_test=False)

2024-04-24 03:15:56,202	WARNING experiment_state.py:323 -- Experiment checkpoint syncing has been triggered multiple times in the last 30.0 seconds. A sync will be triggered whenever a trial has checkpointed more than `num_to_keep` times since last sync or if 300 seconds have passed since last sync. If you have set `num_to_keep` in your `CheckpointConfig`, consider increasing the checkpoint frequency or keeping more checkpoints. You can supress this warning by changing the `TUNE_WARN_EXCESSIVE_EXPERIMENT_CHECKPOINT_SYNC_THRESHOLD_S` environment variable.
2024-04-24 03:15:56,222	INFO tune.py:1042 -- Total run time: 11.07 seconds (7.47 seconds for the tuning loop).


### Using fractional GPUs

In [16]:
config = {
    "objective": "binary:logistic",
    "eval_metric": ["logloss", "error"],
    "tree_method": "gpu_hist",
    "max_depth": tune.randint(1, 9),
    "min_child_weight": tune.choice([1, 2, 3]),
    "subsample": tune.uniform(0.5, 1.0),
    "eta": tune.loguniform(1e-4, 1e-1),
}

tuner = tune.Tuner(
    tune.with_resources(train_breast_cancer, resources={"cpu": 1, "gpu": 0.1}),
    tune_config=tune.TuneConfig(num_samples=10),
    run_config=ray.train.RunConfig(
            storage_path="/home/artifacts/gpu"
        ),
    param_space=config,
)
results = tuner.fit()

2024-04-24 03:17:17,733	WARNING util.py:202 -- The `process_trial_result` operation took 1.115 s, which may be a performance bottleneck.
2024-04-24 03:17:17,734	WARNING util.py:202 -- Processing trial results took 1.117 s, which may be a performance bottleneck. Please consider reporting results less frequently to Ray Tune.
2024-04-24 03:17:17,735	WARNING util.py:202 -- The `process_trial_result` operation took 1.118 s, which may be a performance bottleneck.
2024-04-24 03:17:19,452	WARNING util.py:202 -- The `process_trial_result` operation took 0.870 s, which may be a performance bottleneck.
2024-04-24 03:17:19,456	WARNING util.py:202 -- Processing trial results took 0.874 s, which may be a performance bottleneck. Please consider reporting results less frequently to Ray Tune.
2024-04-24 03:17:19,457	WARNING util.py:202 -- The `process_trial_result` operation took 0.875 s, which may be a performance bottleneck.
2024-04-24 03:17:28,815	WARNING experiment_state.py:323 -- Experiment checkp

### Using full GPUs

In [14]:
print(ray.cluster_resources())

{'CPU': 40.0, 'custom_llm_serving_label': 4.0, 'GPU': 5.0, 'memory': 112660925646.0, 'object_store_memory': 49694317362.0, 'accelerator_type:A10G': 5.0, 'node:10.244.24.11': 1.0, 'node:10.244.21.11': 1.0, 'node:__internal_head__': 1.0, 'node:10.244.23.11': 1.0, 'node:10.244.22.11': 1.0, 'node:10.244.20.11': 1.0}


In [15]:
config = {
    "objective": "binary:logistic",
    "eval_metric": ["logloss", "error"],
    "tree_method": "gpu_hist",
    "max_depth": tune.randint(1, 9),
    "min_child_weight": tune.choice([1, 2, 3]),
    "subsample": tune.uniform(0.5, 1.0),
    "eta": tune.loguniform(1e-4, 1e-1),
}

tuner = tune.Tuner(
    tune.with_resources(train_breast_cancer, resources={"cpu": 8, "gpu": 1}),
    tune_config=tune.TuneConfig(num_samples=10),
    run_config=ray.train.RunConfig(
            storage_path="/home/artifacts/gpufull"
        ),
    param_space=config,
)
results = tuner.fit()

2024-04-24 03:16:28,257	WARNING experiment_state.py:323 -- Experiment checkpoint syncing has been triggered multiple times in the last 30.0 seconds. A sync will be triggered whenever a trial has checkpointed more than `num_to_keep` times since last sync or if 300 seconds have passed since last sync. If you have set `num_to_keep` in your `CheckpointConfig`, consider increasing the checkpoint frequency or keeping more checkpoints. You can supress this warning by changing the `TUNE_WARN_EXCESSIVE_EXPERIMENT_CHECKPOINT_SYNC_THRESHOLD_S` environment variable.
2024-04-24 03:16:28,266	INFO tune.py:1042 -- Total run time: 16.57 seconds (13.40 seconds for the tuning loop).
